Objective:
Do work-flow in model

# pystdf

In [2]:
!pip install pystdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 455.6 kB/s eta 0:00:00a 0:00:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import numpy as np
import pandas as pd
from pystdf.IO import Parser
from pystdf.Writers import TextWriter

In [4]:
fname="/Users/jianlin/Documents/Project_STDF_wafermap/main_Lot_1_Wafer_1_Oct_13_09h33m41s_STDF"

In [5]:
import numpy as np
import pandas as pd
from pystdf.IO import Parser
from pystdf.Writers import TextWriter

class MemoryWriter:
    def __init__(self):
        self.data = []
    def after_send(self, dataSource, data):
        self.data.append(data)
    def write(self,line):
        self.data.append(line)
    def flush(self):
        pass # Do nothing

def ImportSTDF(fname):
    with open(fname,'rb') as fin:
        p = Parser(inp=fin)
        storage = MemoryWriter()
        p.addSink(storage)
        p.parse()
    return storage.data

def STDF2Text(fname,delimiter='|'):
    """ Convert STDF to a list of text representation
    """
    with open(fname,'rb') as fin:
        p = Parser(inp=fin)
        storage = MemoryWriter()
        p.addSink(TextWriter(storage,delimiter=delimiter))
        p.parse()
        return storage.data
    return None

def STDF2Dict(fname):
    """ Convert STDF to a list of dictionary objects
    """
    data = ImportSTDF(fname)
    data_out = []
    for datum in data:
        datum_out = {}
        RecType = datum[0].__class__.__name__.upper()
        datum_out['RecType'] = RecType
        for k,v in zip(datum[0].fieldMap,datum[1]):
            datum_out[k[0]] = v
        data_out.append(datum_out)
    return data_out

def STDF2DataFrame(fname):
    """ Convert STDF to a dictionary of DataFrame objects
    """
    data = ImportSTDF(fname)
    BigTable = {}
    for datum in data:
        RecType = datum[0].__class__.__name__.upper()
        if RecType not in BigTable.keys():
            BigTable[RecType] = {}
        Rec = BigTable[RecType]
        for k,v in zip(datum[0].fieldMap,datum[1]):
            if k[0] not in Rec.keys():
                Rec[k[0]] = []
            Rec[k[0]].append(v)
    for k,v in BigTable.items():
        BigTable[k] = pd.DataFrame(v)
    return BigTable

In [8]:
STDF2Text(fname)

['FAR|2|4\n',
 'ATR|1476351224|dataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15\n',
 'MIR|17:33:30 13-Oct-2016|17:33:44 13-Oct-2016|1| |N| |65535| |1||localhost.localdomain|93000-SOC|cp_32|||vincent|93000|s/w rev. 7.1.4.3 (E), 15-Oct-12\n||25.25||||||||||||||||||\n',
 'SDR|1|2|1|1||||||||||||||||\n',
 'PMR|1|1|Xin|10101|1|1|1\n',
 'PMR|2|2|Xout|10102|2|1|1\n',
 'PMR|3|0|VCC|11||1|1\n',
 'WCR|0.0|0.0|0.0|0| |-32768|-32768| | \n',
 'WIR|1|2|1476351224|\n',
 'PIR|1|1\n',
 'PMR|1|1|Xin|10101|1|1|1\n',
 'PMR|2|2|Xout|10102|2|1|1\n',
 'PMR|3|0|VCC|11||1|1\n',
 'PRR|1|1|0|0|1|1|-5|-4|515|1|0.000|[]\n',
 'PIR|1|1\n',
 'PRR|1|1|0|0|1|1|-1|-4|1|2|1.750|[]\n',
 'PIR|1|1\n',
 'PRR|1|1|0|0|1|1|4|-4|1|3|2.201|[]\n',
 'PIR|1|1\n',
 'PRR|1|1|0|0|1|1|6|-4|1|4|2.803|[]\n',
 'PIR|1|1\n',
 'PRR|1|1|0|0|1|1|7|-4|1|5|2.547|[]\n',
 'PIR|1|1\n',
 'PRR|1|1|0|0|1|1|-7|-3|1|6|2.702|[]\n',
 'PIR|1|1\n',
 'PRR|1|1|0|0|1|1|-3|-3|1|7|2.700|[]\n',
 '

In [62]:
read_file = STDF2DataFrame(fname)
read_file

{'FAR':    CPU_TYPE  STDF_VER
 0         2         4,
 'ATR':       MOD_TIM                                           CMD_LINE
 0  1476351224  dataformatter tester config; Ver: SmarTest s/w...,
 'MIR':       SETUP_T     START_T  STAT_NUM MODE_COD RTST_COD PROT_COD  BURN_TIM  \
 0  1476351210  1476351224         1                 N              65535   
 
   CMOD_COD LOT_ID PART_TYP  ... OPER_FRQ SPEC_NAM SPEC_VER FLOW_ID SETUP_ID  \
 0               1           ...                                               
 
   DSGN_REV ENG_ID ROM_COD SERL_NUM SUPR_NAM  
 0                                            
 
 [1 rows x 38 columns],
 'SDR':    HEAD_NUM  SITE_GRP  SITE_CNT SITE_NUM HAND_TYP HAND_ID CARD_TYP CARD_ID  \
 0         1         2         1      [1]                                     
 
   LOAD_TYP LOAD_ID DIB_TYP DIB_ID CABL_TYP CABL_ID CONT_TYP CONT_ID LASR_TYP  \
 0                                                                              
 
   LASR_ID EXTR_TYP EXTR_ID  


# parse step by step

In [67]:
#read and parsing
with open(fname,'rb') as fin:
    p = Parser(inp=fin)
    storage = MemoryWriter()
    p.addSink(storage)
    p.parse()
    for line in storage.data:
        print(line)

(<pystdf.V4.Far object at 0x7ff75f8c0a90>, [2, 4])
(<pystdf.V4.Atr object at 0x7ff75f8c0130>, [1476351224, 'dataformatter tester config; Ver: SmarTest s/w rev. 7.1.4.3 (E), 15-Oct-12; dataformatter 2.0.9 Oct 15 2012 18:37:15'])
(<pystdf.V4.Mir object at 0x7ff75f8c0c10>, [1476351210, 1476351224, 1, ' ', 'N', ' ', 65535, ' ', '1', '', 'localhost.localdomain', '93000-SOC', 'cp_32', '', '', 'vincent', '93000', 's/w rev. 7.1.4.3 (E), 15-Oct-12\n', '', '25.25', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''])
(<pystdf.V4.Sdr object at 0x7ff75f6ceb50>, [1, 2, 1, [1], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''])
(<pystdf.V4.Pmr object at 0x7ff75f6e5d30>, [1, 1, 'Xin', '10101', '1', 1, 1])
(<pystdf.V4.Pmr object at 0x7ff75f6e5d30>, [2, 2, 'Xout', '10102', '2', 1, 1])
(<pystdf.V4.Pmr object at 0x7ff75f6e5d30>, [3, 0, 'VCC', '11', '', 1, 1])
(<pystdf.V4.Wcr object at 0x7ff75f616940>, [0.0, 0.0, 0.0, 0, ' ', -32768, -32768, ' ', ' '])
(<pystdf.V4.Wir obj

In [79]:
#to dataframe
data = ImportSTDF(fname)
for datum in data:
    if datum[0].__class__.__name__.upper()=='PRR':
        print(datum[0])

In [80]:
#to dataframe
data = ImportSTDF(fname)
for datum in data:
    if datum[0].__class__.__name__.upper()=='PRR':
        print(datum[0].fieldMap,"\n")

(('HEAD_NUM', 'U1'), ('SITE_NUM', 'U1'), ('PART_FLG', 'B1'), ('NUM_TEST', 'U2'), ('HARD_BIN', 'U2'), ('SOFT_BIN', 'U2'), ('X_COORD', 'I2'), ('Y_COORD', 'I2'), ('TEST_T', 'U4'), ('PART_ID', 'Cn'), ('PART_TXT', 'Cn'), ('PART_FIX', 'Bn')) 

(('HEAD_NUM', 'U1'), ('SITE_NUM', 'U1'), ('PART_FLG', 'B1'), ('NUM_TEST', 'U2'), ('HARD_BIN', 'U2'), ('SOFT_BIN', 'U2'), ('X_COORD', 'I2'), ('Y_COORD', 'I2'), ('TEST_T', 'U4'), ('PART_ID', 'Cn'), ('PART_TXT', 'Cn'), ('PART_FIX', 'Bn')) 

(('HEAD_NUM', 'U1'), ('SITE_NUM', 'U1'), ('PART_FLG', 'B1'), ('NUM_TEST', 'U2'), ('HARD_BIN', 'U2'), ('SOFT_BIN', 'U2'), ('X_COORD', 'I2'), ('Y_COORD', 'I2'), ('TEST_T', 'U4'), ('PART_ID', 'Cn'), ('PART_TXT', 'Cn'), ('PART_FIX', 'Bn')) 

(('HEAD_NUM', 'U1'), ('SITE_NUM', 'U1'), ('PART_FLG', 'B1'), ('NUM_TEST', 'U2'), ('HARD_BIN', 'U2'), ('SOFT_BIN', 'U2'), ('X_COORD', 'I2'), ('Y_COORD', 'I2'), ('TEST_T', 'U4'), ('PART_ID', 'Cn'), ('PART_TXT', 'Cn'), ('PART_FIX', 'Bn')) 

(('HEAD_NUM', 'U1'), ('SITE_NUM', 'U1'), ('PART_